In [20]:
import tensorflow_datasets as tfds
import tensorflow as tf

import os

# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [21]:
print(tf.__version__)

2.9.1


In [22]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets['train'], datasets['test']

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  25%|██▌       | 1/4 [00:01<00:03,  1.03s/ url]

Dl Completed...:  75%|███████▌  | 3/4 [00:01<00:00,  2.31 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:01<00:00,  2.51 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:02<00:00,  1.85 url/s]
                                                                        

Dataset mnist downloaded and prepared to C:\Users\Asus\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [23]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [24]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


In [25]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [28]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

In [29]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [31]:
# Define the checkpoint directory to store the checkpoints.
checkpoint_dir = '../training_checkpoints'
# Define the name of the checkpoint files.
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [32]:
# Define a function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [33]:
# Define a callback for printing the learning rate at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(        epoch + 1, model.optimizer.lr.numpy()))

In [34]:
# Put all the callbacks together.
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [35]:
EPOCHS = 12

model.fit(train_dataset, epochs=EPOCHS, callbacks=callbacks)

Epoch 1/12
938/938 [==============================] - ETA: 0s - loss: 0.4524 - accuracy: 0.8418
Learning rate for epoch 1 is 0.0010000000474974513
938/938 [==============================] - 6s 4ms/step - loss: 0.4524 - accuracy: 0.8418 - lr: 0.0010
Epoch 2/12
931/938 [============================>.] - ETA: 0s - loss: 0.3040 - accuracy: 0.8918
Learning rate for epoch 2 is 0.0010000000474974513
938/938 [==============================] - 4s 4ms/step - loss: 0.3038 - accuracy: 0.8919 - lr: 0.0010
Epoch 3/12
933/938 [============================>.] - ETA: 0s - loss: 0.2628 - accuracy: 0.9059
Learning rate for epoch 3 is 0.0010000000474974513
938/938 [==============================] - 3s 4ms/step - loss: 0.2628 - accuracy: 0.9058 - lr: 0.0010
Epoch 4/12
930/938 [============================>.] - ETA: 0s - loss: 0.2149 - accuracy: 0.9230
Learning rate for epoch 4 is 9.999999747378752e-05
938/938 [==============================] - 3s 4ms/step - loss: 0.2152 - accuracy: 0.9230 - lr: 1.0000e-04


In [36]:
%tensorboard --logdir=logs